<a href="https://colab.research.google.com/github/rneyns/GRM1_public/blob/master/Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!rm -rf GRM1
!git clone --single-branch --depth=1 --branch master https://github.com/rneyns/GRM1_public
!pip3 uninstall seaborn
!pip3 install seaborn==0.11.0

#GRM1 practical

##Preliminary data analysis

First we will load and visualise the composite sattelite image and the data-set that were created in the previous part of the exercise.

In [ ]:
#import the necessary packages
import numpy as np
import seaborn as sns
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
from GRM1_public.Utility_functions import *

from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score

### Training data

In [ ]:
#load the data
data = pd.read_csv('INSERT PATH HERE')

#print the first 5 entries of the dataframe
data.head()

In [ ]:
#remove the outliers
data = data[data['b_1'] <= 18000] #drop the outliers caused by atmospheric correction
data = data[data['b_5'] <= 18000]
data = data[data['b_6'] <= 18000]

The following lines of code allow you to plot the distribution of the different classes with respect to the different bands. Additionally, a scatterplot is created for each pair of bands.

To improve the visualisation for the report, it possible to only plot the classes/bands that show the most interesting patters. To limit the number of bands that is included in the visualisation, adjust the data parameter. More specifically, exclude the names of the columns that you wish to leave out.


In [ ]:
#plot the distribution of the training data
sns.pairplot(data=data[['id','b_1','b_2','b_3','b_4','b_5','b_6']],
             kind="scatter", hue="id", plot_kws=dict(s=20, edgecolor="white", linewidth=1))


In [ ]:
#The visualisation of the distribution over 1 band can also be made separately
fig, ax = plt.subplots(figsize=(15,10))
sns.kdeplot(data=data, x='b_2', hue="id",ax=ax)

In [ ]:
#Or even one band and one class
fig,ax = plt.subplots(figsize=(15,10))

data_vis = data.loc[data['id'] == 1] #Specify the class you want to visualise
sns.histplot(data = data_vis,x = 'b_2', kde=True,ax=ax) #Specify the band you want to visualise

In [ ]:
#Finally, the scatterplots can also be shown in isolation
fig, ax = plt.subplots(figsize=(15,10))
sns.scatterplot(data=data, x="b_2", y="b_3", hue="id",ax=ax)

In [ ]:
#Visualisation of the spectral signatures

#Drop unncessary columns when needed, the dataframe should only contain the class identifier and the band values
data_vis = data
#data_vis = data.drop(['UNNECESSARY COLUMN'], axis=1)
 

#Take the mean value for each value per class and plot it
data_mean = data_vis.groupby('id').mean() 
plot_spectral_signature(data_mean)

### ROI data

In [ ]:
#import the gdal python package
import gdal

#load the GeoTIFF file (composite sattelite image)
raster_file = 'INSERT PATH HERE' 
driverTiff = gdal.GetDriverByName('GTiff') 
raster = gdal.Open(raster_file) 

In [ ]:
#convert the gdal raster file to a pandas dataframe, this makes it easier to handle
nbands = raster.RasterCount 
data_raster = np.empty((raster.RasterXSize*raster.RasterYSize, nbands))

for i in range(1, nbands+1): 
    band = raster.GetRasterBand(i).ReadAsArray() 
    data_raster[:, i-1] = band.flatten()

data_raster = pd.DataFrame(data_raster)
data_raster.columns = ['b1','b2','b3','b4','b5','b7']


data_raster.head()

Now you will plot a pairplot for the full raster, so you can compare the distribution with your training-set. I advice you to save this image once the visualisation is generated so you don't have to run it again (it takes some time). 

In [ ]:
#remove the outliers before visualisation
data_raster_vis = data_raster[data_raster[ ['b1','b2','b3','b4','b5','b7']] <= 18000]

sns.pairplot(data=data_raster_vis,
             kind="scatter", plot_kws=dict(s=40, edgecolor="white", linewidth=0.2))

##Classification

You will train three classification algorithms on our data-set: a K-means classifier, a maximum likelihood classifier and a classifier of your choice. For each classifier, we will make use of the overall accuracy, the confusion matrix and the kappa-value to assess the results. Based on the confusion matrix you can subsequently derive the other accuracy measures we ask for in the report. 



In [ ]:
#load validation data
val = pd.read_csv('INSERT PATH HERE')
val = val[val['b_1'] <= 18000] #drop the outliers caused by atmospheric correction
val = val[val['b_5'] <= 18000] 
val = val[val['b_6'] <= 18000] 
val.head()

In [ ]:
#split both sets in input and label sets, the data is also transformed to a numpy array so it can be used in scikit-learn
train_x = data[['b_1','b_2','b_3','b_4','b_5','b_6']].values
train_y = data[['id']].values

val_x = val[['b_1','b_2','b_3','b_4','b_5','b_6']].values
val_y = val[['id']].values

In [ ]:
#change datatypes of the b_columns if necessary
val[['b_1','b_2','b_3','b_4','b_5','b_6']] = val[['b_1','b_2','b_3','b_4','b_5','b_6']].astype(float)
data[['b_1','b_2','b_3','b_4','b_5','b_6']] = data[['b_1','b_2','b_3','b_4','b_5','b_6']].astype(float)

### Minimum distance to means classifier

In [ ]:
#Initialise and train the model
clf1 = NearestCentroid()
clf1.fit(train_x,train_y)

#### Accuracy assessment
Below, you will use three metrics to assess the performance of your model on the validation set:

- Overall accuracy
- The confusion matrix
- Cohen's Kappa value

Based on the confusion matrix you are also expected to calculate the users and producers accuracy. 

In [ ]:
#apply the classifier to the training and validation set and assess the overall accuracy
predictions1 = clf1.predict(train_x)
acc_train = accuracy_score(train_y,predictions1)

predictions_val1 = clf1.predict(val_x)
acc_val = accuracy_score(val_y,predictions_val1)

print('The overall accuracy score is %.3f for the training set and %.3f on the validation set' % (acc_train,acc_val))

In [ ]:
#now plot the normalized confusion matrix 

fig, ax = plt.subplots(figsize=(10, 10))
disp = plot_confusion_matrix(clf1, val_x, val_y,
                                  cmap=plt.cm.Blues,
                                  normalize=None,ax=ax,values_format=".0f")
disp.ax_.set_title(("Confusion matrix"))
plt.show()

In [ ]:
#next, calculate the kappa-value

Kappa = cohen_kappa_score(predictions_val1, val_y)
print('The Kappa value for your classification is: %.3f for the validation set' % (Kappa))

The final block of code will export the result of the classification to a GeoTIFF file which can be opened in QGIS. Use this to make a nice map/visualisation of your results. 

Once the code has been run, the geotiff will become available in the 'files' section on the left (it might be necessary to refresh first).

In [ ]:
#export the result to GeoTIFF
to_GeoTIFF(out_name = "dist_mean.tif", clf = clf1, data = data_raster, raster = raster)

### Maximum likelihood classification

In [ ]:
#Initialise and train the model
clf2 = GaussianNB()
clf2.fit(train_x,train_y)

#### Accuracy assessment

In [ ]:
#apply the classifier to the training and validation set and assess the overall accuracy
predictions2 = clf2.predict(train_x)
acc_train = accuracy_score(train_y,predictions2)

predictions_val2 = clf2.predict(val_x)
acc_test = accuracy_score(val_y,predictions_val2)

print('The overall accuracy score is %.3f for the training set and %.3f on the validation set' % (acc_train,acc_val))

In [ ]:
#now plot the normalized confusion matrix 
fig, ax = plt.subplots(figsize=(10, 10))
disp = plot_confusion_matrix(clf2, val_x, val_y,
                                  cmap=plt.cm.Blues,
                                  normalize=None,ax=ax,values_format=".0f")
disp.ax_.set_title(("Confusion matrix"))
plt.show()

In [ ]:
#next, calculate the kappa-value
Kappa = cohen_kappa_score(predictions_val2, val_y)
print('The Kappa value for your classification is: %.3f for the validation set' % (Kappa))

In [ ]:
to_GeoTIFF(out_name = "ML.tif", clf = clf2, data = data_raster, raster = raster)

### Third classifiction agorithm
Throughout the third session for exercise 1 you are expected to implement a third classification algorithm that you have encountered during the lectures. To do this you will use the same python package that was used for the first 2 classification algorithms: "scikit-learn". 
Extensive documentation on the different algorithms available within this package can be found at: https://scikit-learn.org/stable/supervised_learning.html

An overview of the possible algorithms can be seen in the following table:

Algorithm | Documentation
---|---
Artificial neural network | https://scikit-learn.org/stable/modules/neural_networks_supervised.html
Decision tree classifier | https://scikit-learn.org/stable/modules/tree.html
Random forest | https://scikit-learn.org/stable/modules/ensemble.html#forest
K-Nearest neighbour | https://scikit-learn.org/stable/modules/neighbors.html



Below I provide you some code on how to set-up a classification algorithm, fill in the necessary parameters/variables. An example on how to import a specific classifier can always be found in the documentation.  

HINT: an artificial neural network is also often referred to as a multi-layer perceptron.

In [ ]:
#replace ALGORITHM by the algorithm you have selected and PACKAGE by the package in which it can be found.
#A full example of the import code for each algorithm can be found in the documentation of sklearn
from sklearn.PACKAGE import ALGORITHM

#Define the classifier 
clf3 = ALGORITHM() #fill in algorithm by the algorithm you just imported (don't forget the brackets)

#Train the classifier on the training set
clf3.fit(train_x,train_y)

#### Accuracy assessment

In [ ]:
#apply the classifier to the training and validation set and assess the overall accuracy
predictions3 = clf3.predict(train_x)
acc_train = accuracy_score(train_y,predictions3)

predictions_val3 = clf3.predict(val_x)
acc_val = accuracy_score(val_y,predictions_val3)

print('The overall accuracy score is %.3f for the training set and %.3f on the validation set' % (acc_train,acc_val))

In [ ]:
#now plot the normalized confusion matrix 
fig, ax = plt.subplots(figsize=(10, 10))
disp = plot_confusion_matrix(clf3, val_x, val_y,
                                  cmap=plt.cm.Blues,
                                  normalize=None,ax=ax,values_format=".0f")
disp.ax_.set_title(("Confusion matrix"))
plt.show()

In [ ]:
#next, calculate the kappa-value
Kappa = cohen_kappa_score(predictions_val3, val_y)
print('The Kappa value for your classification is: %.3f for the validation set' % (Kappa))

In [ ]:
to_GeoTIFF(out_name = "ML.tif", clf = clf3, data = data_raster, raster = raster)